In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

dataGen = pd.read_excel('inputs.xlsx', sheet_name = 'gen')
dataLoad = pd.read_excel('inputs.xlsx', sheet_name = 'load')

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
dataGen

,id,limit,cost
0,0,20,0.10
1,1,10,0.05
2,2,40,0.30
3,3,50,0.40
4,4,5,0.01


In [3]:
dataLoad

,id,value
0,0,50
1,1,20
2,2,30


In [4]:
dataGen.cost[1]

0.05

In [5]:
Ng = len(dataGen)

In [6]:
model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng),bounds = (0,None))

In [7]:
model.Pg.pprint()

Pg : Size=5, Index=Pg_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :  None : False :  True :  Reals
      1 :     0 :  None :  None : False :  True :  Reals
      2 :     0 :  None :  None : False :  True :  Reals
      3 :     0 :  None :  None : False :  True :  Reals
      4 :     0 :  None :  None : False :  True :  Reals


In [8]:
Pg = model.Pg

pg_sum = sum([Pg[g] for g in dataGen.id])
print(pg_sum)

model.lim = pyo.ConstraintList()
for k in dataGen.id:
    model.lim.add( expr = Pg[k] <= dataGen.limit[k])
model.lim.pprint()

model.balance = pyo.Constraint(expr = pg_sum == sum(dataLoad.value))
model.balance.pprint()

model.cond = pyo.Constraint(expr = Pg[0] + Pg[3] >= dataLoad.value[0])
model.cond.pprint()

Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4]
lim : Size=5, Index=lim_index, Active=True
    Key : Lower : Body  : Upper : Active
      1 :  -Inf : Pg[0] :  20.0 :   True
      2 :  -Inf : Pg[1] :  10.0 :   True
      3 :  -Inf : Pg[2] :  40.0 :   True
      4 :  -Inf : Pg[3] :  50.0 :   True
      5 :  -Inf : Pg[4] :   5.0 :   True
balance : Size=1, Index=None, Active=True
    Key  : Lower : Body                                  : Upper : Active
    None : 100.0 : Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] : 100.0 :   True
cond : Size=1, Index=None, Active=True
    Key  : Lower : Body          : Upper : Active
    None :  50.0 : Pg[0] + Pg[3] :  +Inf :   True


In [13]:
obj_sum = sum([Pg[g]*dataGen.cost[g] for g in dataGen.id])
model.obj = pyo.Objective(expr = obj_sum, sense=minimize)

opt = SolverFactory('gurobi')
results = opt.solve(model)

dataGen['Pg'] = [pyo.value(Pg[g]) for g in dataGen.id]
print(dataGen)

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
   id  limit  cost    Pg
0   0     20  0.10  20.0
1   1     10  0.05  10.0
2   2     40  0.30  35.0
3   3     50  0.40  30.0
4   4      5  0.01   5.0
